<a href="https://colab.research.google.com/github/dskyshr/cadaSimulator/blob/main/my_module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Create preference table of my model (Each thier sum of utiliries is equal to 1.)
def create_preference_table():

    pref_odnl = []
    pref_cdnl = []
    pref_cdnl_ge = []

    # Image of preferences (When number of school is 5 and number of student is 100)
    #
    # i = 0  1  2  3 ...19 20 21 ... 39 40 41 ... 59 60 61 ... 79 80 81 ... 97 98 99
    # ------------------------------------------------------------------------------
    # a = 0  0  0  0 ... 0  0  0 ...  0  0  0 ...  0  0  0 ...  0  0  0 ...  0  0  0
    # a = -  -  -  -     -  1  1 ...  1  1  1 ...  1  1  1 ...  1  1  1 ...  1  1  1
    # a = -  -  -  - ... -  -  - ...  -  2  2 ...  2  2  2 ...  2  2  2 ...  2  2  2
    # a = -  -  -  - ... -  -  - ...  -  -  - ...  -  3  3 ...  3  3  3 ...  3  3  3
    # a = -  -  -  - ... -  -  - ...  -  -  - ...  -  -  - ...  -  4  4 ...  4  4  4

    # Common preference intensity for each student
    u_school = []
    for _ in range(num_school):
        u_school.append(random.random())

    # Create preference for each student
    for i in range(num_student):

        pref_i_odnl = []
        pref_i_cdnl = []

        if randomize_acceptable == True :
            # Lottery of number of preferences
            num_pref_i = random.randint(1,num_school)

            for j,u_a in enumerate(u_school):
                # Individual preference intensity for school a
                u_ia = random.random()
                v_tilde_ia = (alpha*u_a) + ((1-alpha)*u_ia)
                pref_i_odnl.append(j)
                pref_i_cdnl.append(v_tilde_ia)
                # If j reaches to number of lottery, break.
                if j+1 >= num_pref_i:
                    break
        else :
            for j,u_a in enumerate(u_school):
                # Individual preference intensity for school a
                u_ia = random.random()
                v_tilde_ia = (alpha*u_a) + ((1-alpha)*u_ia)
                pref_i_odnl.append(j)
                pref_i_cdnl.append(v_tilde_ia)
                # Regurate the number of preferences for i within the quotient of the number of students divided by the number of schools.
                if j > (i / int(num_student / num_school)) - 1:
                    break

        # Randomize ordinally (option)
        if randomize_order:
            random.shuffle(pref_i_odnl)
            print('randomize') if debug else None
        else:
            pref_i_odnl.sort(reverse=False) # Rearrenge as Ascending order

        pref_odnl.append(pref_i_odnl)
        pref_i_cdnl.sort(reverse=True) # Rearrenge as descent order
        pref_cdnl.append(pref_i_cdnl)

        # Change the sum of each utility to 1 (option)
        if normalize_utils == True:
            #print("Normarize!")
            pref_i_cdnl_ge = []
            total_util_i = sum(pref_i_cdnl)
            for v in pref_i_cdnl:
                v = v / total_util_i
                pref_i_cdnl_ge.append(v)
            pref_cdnl_ge.append(pref_i_cdnl_ge)

    if normalize_utils == True:
        pref_cdnl = pref_cdnl_ge

    return pref_odnl, pref_cdnl


def get_competitors():

    competitors = []
    for a in range(num_school):
        competitors_a = []
        for i in range(num_student):
            # When the element is exists and number of 'pref' > j
            if a in pref_odnl[i]:
                competitors_a.append(i)
        competitors.append(competitors_a)

    return competitors


# Culcurate expected utilities for each student amang the fixed combination.

# Not use
def culcurate_any_expected_utilities_old(target_comb):

    competitors_origin = get_competitors()

    # Verify the competitors of the targeted school.
    competitors_c = [0] * num_school
    for t in target_comb :
        competitors_c[t] += 1

    # Culcurate expected utilities.
    eu_c = []
    for i,pref_odnl_i in enumerate(pref_odnl):
        eu_i = 0
        for a,school in enumerate(pref_odnl_i):
            # i's utility value for school a.
            util_ij = pref_cdnl[i][a]
            #print(f"util_ij:{util_ij}")
            ##################### This calculation is incorrect ###############################
             # When school a is her target, the number of competitors is the number of students who target school a.
            if school == target_comb[i]:
                # Even though school a is undersubscribed, the probability of enrollment never surpass 1.
                eu_i += (util_ij/competitors_c[school]) * min(num_capacity,competitors_c[school])
            # When nobady targets school a, the number of competitors is the number of student who prefer school a without target school.
            else:
                if school not in target_comb:
                    # Even though school a is undersubscribed, the probability of enrollment never surpass 1.
                    eu_i += (util_ij/len(competitors_origin[school])) * min(num_capacity,len(competitors_origin[school]))
                else : # Otherwise, this school is inaccetable.
                    continue
        eu_c.append(eu_i)

    return eu_c


def culcurate_any_expected_utilities(target_comb):

    students = [i for i in range(num_student)]
    schools  = [a for a in range(num_school)]

    # preference
    #pref_odnl = [
    #    [0, 1],
    #    [0, 1, 2, 3],
    #    [0, 1, 2, 3],
    #    [0, 1, 2, 3],
    #]

    targets = {school: [] for school in schools}
    for i, a in enumerate(target_comb):
        if a in targets:
            targets[a].append(i)

    # Capacities
    L = {school: num_capacity for school in schools}

    # Number of attemptions for Monte Carlo
    #N_sim = 100000
    N_sim = 1000
    counter = defaultdict(lambda: defaultdict(int))

    for _ in range(N_sim):
        # Decide π
        pi = students.copy()
        random.shuffle(pi)

        # Create the priority based on Pi
        school_priority = {}
        for a in schools:
            #print(f"g = {g}")
            target_group     = [i for i in pi if i     in targets[a]]
            non_target_group = [i for i in pi if i not in targets[a]]
            school_priority[a] = target_group + non_target_group

        # Provisional list
        provisional = {a: [] for a in schools}

        # Students' state
        remaining_prefs = {i: deque(pref_odnl[i]) for i in students}
        free_students = set(students)

        while free_students:
            i = free_students.pop()
            if not remaining_prefs[i]:
                continue
            a = remaining_prefs[i].popleft()

            # Acception temporarilly
            provisional[a].append(i)

            # Rejection when capacity over
            priority_order = school_priority[a]
            provisional[a].sort(key=lambda x: priority_order.index(x))
            rejected = provisional[a][L[a]:]
            provisional[a] = provisional[a][:L[a]]

            # Re-applying by refused student
            for r in rejected:
                free_students.add(r)

        # Count probabillity
        for a in schools:
            for i in provisional[a]:
                counter[i][a] += 1

    # Calculate the allocation
    Phi = [ [counter[i][a] / N_sim for a in schools] for i in students]

    # Calculate the expected utiliry
    eu_c = [ 0 for a in students ]
    for i,pref_cdnl_i in enumerate(pref_cdnl):
        for a, pref_cdnl_i_a in enumerate(pref_cdnl_i):
            eu_c[i] += (Phi[i][a] * pref_cdnl_i_a)

    return eu_c


# Find CADA equilibrium by Best-Response Dynamics
def find_cada_equilibrium_by_BRD():

    # If we want to bifurcate proccess by option parameters,
    # receive arguments and write proccess here
    #if option['option_name'] == 'option_name':
    #    process something

    # Take an arbitrary combination.
    target_comb = []
    for pref_odnl_i in pref_odnl:
        target_comb.append(pref_odnl_i[0])

    # Culcurate EU of this combination.
    #eu_old = culcurate_any_expected_utilities_old(target_comb)
    eu = culcurate_any_expected_utilities(target_comb)
    print(f"Initial eu:{eu}") if debug else None

    # Set up of initial student.
    i = 0
    max_eu = [ 0 for _ in range(num_student) ]
    max_eu[i] = eu[i] # initial value

    # Converge flag
    converge_seq = 0

    while converge_seq < num_student :
        max_tc_i = target_comb.copy()

        # Moving flag
        moved = False

        # Elements of "pref_odnl[i]"" are target candidates for student i
        for a in pref_odnl[i]:

            target_comb = target_comb[:i] + [a] + target_comb[i+1:]
            #print(f"  target_comb = {target_comb}") if debug else None
            this_eu = culcurate_any_expected_utilities(target_comb)
            #print(f"  this_eu = {this_eu}") if debug else None

            if this_eu[i] > max_eu[i]:
                #print(f"    Updating...") if debug else None
                if max_tc_i != target_comb :
                    moved = True
                # Update
                max_tc_i = target_comb.copy()
                max_eu[i] = this_eu[i]

        if moved == False :
            # If she didn't move, record it.
            converge_seq += 1
        else :
            # Otherwise, reset converge flag.
            converge_seq = 0

        target_comb = max_tc_i

        # Advance to next student.
        if i == num_student - 1:
            i = 0
        else:
            i += 1

    return max_tc_i, culcurate_any_expected_utilities(max_tc_i)

    # ACY(2015)Therorem3よりCADAは純粋戦略均衡が必ず存在する
    # 時間があれば拡張用に純粋戦略均衡が存在しない場合のエラー処理も作る


# Create CADA priority by DA-STB (1 equilibrium version. Using random.shuffle).
def create_CADA_priority() :

    base_t = list(range(num_student))
    base_r = list(range(num_student))
    random.shuffle(base_t)
    random.shuffle(base_r)

    g = []
    for a in range(num_school):
        competitors_t_a = base_t.copy() # For target lottery
        competitors_r_a = base_r.copy() # For regular lottery

        for i,pref_odnl_i in enumerate(pref_odnl):
            if a in pref_odnl_i:
                if a == eq[i]: # If a is i's target
                    competitors_r_a.remove(i)
                else:
                    competitors_t_a.remove(i)
            else:
                competitors_t_a.remove(i)
                competitors_r_a.remove(i)

        # Combine
        competitors_a = competitors_t_a + competitors_r_a

        # Add to parent array
        g.append(competitors_a)

    return g


# Create CADA priority by DA-MTB (1 equilibrium version. Using random.shuffle).
def create_CADA_priority_by_damtb() :

    #eq = br_tc[0] # After we considered about Eq, only Eq remains in tc.
    print(f'Equilibrium: {eq}') if debug else None
    competitors_t = []
    competitors_r = []

    for a in range(num_school):
        competitors_t_a = [] # For target lottery
        competitors_r_a = [] # For regular lottery
        for i in range(num_student):
            # When there exists the element and number of 'pref' > j
            if a == eq[i]:
                competitors_t_a.append(i)
            else:
                if a in pref_odnl[i]:
                    competitors_r_a.append(i)
        # Shuffle
        if len(competitors_t_a) >= 2:
            random.shuffle(competitors_t_a)
        if len(competitors_r_a) >= 2:
            random.shuffle(competitors_r_a)
        # Add to parent array
        competitors_t.append(competitors_t_a)
        competitors_r.append(competitors_r_a)

    #competitors_t.sort(reverse=False)
    #competitors_t.sort(reverse=False)

    g = [] # g (in G) is a combination of the priority (permutation) for all schools.
    # Set the permitations to g
    for i,competitors_t_a in enumerate(competitors_t):
        # Choose a permutation randomly
        perms = competitors_t_a + competitors_r[i]
        g.append(perms)

    return g


# Implementation DA algorithm. (Multiple seats version.)
def implement_DA():

    # Initialize match with empty arrays
    match     = [[] for _ in range(num_school)]
    unmatched = list(range(num_student))

    while len(unmatched) > 0:

        # Loop 1 (i's preference)
        for i,pref_i_odnl in enumerate(pref_odnl):

            # If ith student is not in 'unmatched', we don't need to consider about this student anymore.
            if i not in unmatched:
                continue
            # If ith student has no preference, we can delete her from unmatched and go to i+1th student
            if not any(isinstance(school, int) for school in pref_i_odnl):
                print('Student '+str(i) + ' has been removed from unmatched because her preference is empty.') if debug else None
                unmatched.remove(i)
                continue
            # The algorithm is done when unmatched list is empty.
            if len(unmatched) == 0 :
                break

            # Loop 2 (Schools in i's preference)
            for a,school in enumerate(pref_i_odnl):

                # ith student's ath prefere school
                next_i = 0
                # If there is no ath school in her preference, go to school a+1th
                if school == '':
                    continue
                print('student '+ str(i) + '\'s '+ str(a) + 'th choice : school '+ str(school) +'.') if debug else None

                # Loop 3 (Students in a's priority)
                print('- school '+ str(school) + '\'s priority : ' + str(g[school])) if debug else None
                for rank,student in enumerate(g[school]):

                    # The rank of ith student for school a.
                    if student == i :
                        print( '-- current matched student : '+ str(match[school]) ) if debug else None
                        # When non full capacity
                        print('--- quota : '+ str(len(match[school])) + '/'+ str(num_capacity)) if debug else None
                        if len(match[school]) < num_capacity:
                            match[school].append({'student':i,'rank':rank})
                            print ('--- school '+ str(school) + ' accepted student '+ str(i) + '.') if debug else None
                            pref_odnl[i][a] = ''
                            unmatched.remove(i)
                            next_i = 1 # Flag for going to i+1th student
                        # When full capacity.
                        else:
                            lowest = {'rank': num_student} # Default lowest student
                            print('lowest:'+str(lowest)) if debug else None
                            print('match:'+str(match)) if debug else None
                            for m in match[school]:
                                print('m:'+str(m)) if debug else None
                                # Find a student who has lowest rank for school a.
                                if m['rank'] < lowest['rank']:
                                    lowest = m
                            print('lowest:'+ str(lowest)) if debug else None
                            # When the rank of ith studnet is higher than current matched student.
                            if rank < int(lowest['rank']):
                                unmatched.append(lowest['student'])
                                print ('--- school '+ str(school) + ' refused student '+ str(lowest['student']) + '!') if debug else None
                                match[a].remove({'student':lowest['student'],'rank':lowest['rank']})
                                match[a].append({'student':i,'rank':rank})
                                print ('--- school '+ str(school) + ' accepted student '+ str(i) + '.') if debug else None
                                pref_odnl[i][a] = ''
                                unmatched.remove(i)
                                next_i = 1 # Flag for going to i+1th student
                            else:
                                print ('--- school '+ str(school) + ' refused student '+ str(i) + '!!') if debug else None
                                pref_odnl[i][a] = ''
                                print('Go to next prefer school.') if debug else None
                                break # Go to ith student's a+1th prefer school because she was refused by school a

                        break
                if next_i == 1:
                    break
    return match


# Judgement for CAC,CR.
def judge_CAC():
    total_match = 0

    for m in match:
        total_match += len(m)
    cac = 0
    if total_match == num_school * num_capacity:
        cac = 1
    else:
        cac = 0
    # Consumption rate
    cr = total_match/(num_school * num_capacity)

    return cac, cr


# Culcurate ex-ante welfair
def get_exante_welfair(eu):
    welfair = 0
    for eu_i in eu:
        welfair += eu_i
    #print(f'Ex-ante welfair : {welfair}') if debug else None
    return welfair


# Culcurate expected utility for each student (For normal DA-MTB)
def culcurate_expected_utilities():

    #competitors = get_competitors()
    eu = []
    for i,util_i in enumerate(pref_cdnl):
        eu_i = 0
        for a,util_ij in enumerate(util_i):
            eu_i += util_ij/len(competitors[a])
        eu.append(eu_i)

    return eu


# Create random priority for each school (For DA-MTB)
def create_random_priority():

    #competitors.sort(reverse=False)
    #print(f"competitors = {competitors}")

    g = []
    for competitors_a in competitors:
        # Choose a permutation randomly
        random.shuffle(competitors_a)
        g.append(competitors_a)

    return g


# Create random priority for all school (For DA-STB)
def create_random_priority_by_STB():

    base = list(range(num_student))
    random.shuffle(base)

    g = []
    for competitors_a in competitors:
        reordered_competitors_a = []
        for i in base:
            if i in competitors_a:
                reordered_competitors_a.append(i)
        g.append(reordered_competitors_a)

    return g

def print_eq_to_csv():
    header = [
        'num_school',
        'num_capacity',
        'num_student',
        'alpha',
    ]
    data = [
        num_school,
        num_capacity,
        num_student,
        alpha,
    ]
    for i in range(num_school):
        header.append(f"school {i}")
        data.append(eq.count(i))

    # Check if the file exists and write headers if not
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)
        print(f"Created new file and wrote header to {file_path}") # Added for debugging

    # Append the data to the CSV file
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)
        print(f"Appended data to {file_path}") # Added for debugging

    print(f"Finished attempting to write data to {file_path}") # Added for debugging

def print_result_to_csv():

    # Check if the file exists and write headers if not
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([
                'num_school',
                'num_capacity',
                'num_student',
                'alpha',
                'avg_cr_cada',
                'avg_cr_damtb',
                'avg_cr_dastb',
                'gap_cada_damtb',
                'gap_cada_dastb',
            ])
        print(f"Created new file and wrote header to {file_path}") # Added for debugging

    # Append the data to the CSV file
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([
            num_school,
            num_capacity,
            num_student,
            alpha,
            avg_cr_cada,
            avg_cr_damtb,
            avg_cr_dastb,
            gap_cada_damtb,
            gap_cada_dastb,
        ])
        print(f"Appended data to {file_path}") # Added for debugging

    print(f"Finished attempting to write data to {file_path}") # Added for debugging


# Show CSV contents
def show_csv_contents(file_path):
    try:
        df = pd.read_csv(file_path)
        display(df)
    except FileNotFoundError:
        print(f"File '{file_path}' was not found.")
    except Exception as e:
        print(f"A error has occured while reading the file: {e}")


# Download CSV
def download_csv(file_path):
    from google.colab import files
    try:
        files.download(file_path)
        print(f"Attempting to download '{file_path}'...")
    except FileNotFoundError:
        print(f"File '{file_path}' was not found. Can not download.")
    except Exception as e:
        print(f"A error has occured while downloading the file: {e}")


# Delete CSV file
def delete_csv(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"File '{file_path}' was deleted.")
    else:
        print(f"File '{file_path}' does not exist.")